# Bibliotecas

In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import time
import requests
import re
import unidecode
from scrapy import Selector
from pandas import json_normalize
from random_user_agent.user_agent import UserAgent
import random
import sqlite3
import datetime
import pytz
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from urllib.parse import urlencode
import psycopg2
from psycopg2.extras import execute_batch # inserir múltiplos valores
from sqlalchemy import create_engine
import os
# from utils.utils_psql import UtilsPSQL
from utils.utils_scraper import ScraperZap
import concurrent.futures

# Obtendo dados de municípos via API IBGE

In [2]:
# Dados dos Estados
url_uf = 'https://servicodados.ibge.gov.br/api/v1/localidades/estados'

r_uf = requests.get('https://servicodados.ibge.gov.br/api/v1/localidades/estados')
uf_json = r_uf.json()

df_estados = json_normalize(uf_json, sep = '_')

# Dados dos municípios
ids_uf = df_estados['id'].to_list()
string_ids = ''
for i in ids_uf:
    string_ids += '|' + str(i)
    
url_mun = f'https://servicodados.ibge.gov.br/api/v1/localidades/estados/{string_ids[1::]}/municipios'
r_mun = requests.get(url_mun)
mun_json = r_mun.json()
mun_df = json_normalize(mun_json)

# Dataframe final com dados dos municípios
df_municipios = mun_df[['id','nome','microrregiao.mesorregiao.UF.sigla','regiao-imediata.regiao-intermediaria.UF.regiao.nome','microrregiao.mesorregiao.UF.nome']].copy()
df_municipios.rename(
    columns = {
        'id':'id_municipio',
        'nome':'str_municipio',
        'microrregiao.mesorregiao.UF.nome':'str_estado',
        'microrregiao.mesorregiao.UF.sigla':'str_uf',
        'regiao-imediata.regiao-intermediaria.UF.regiao.nome':'str_regiao'
    },
    inplace = True
)
df_municipios['str_local'] = df_municipios['str_uf'].apply(lambda x: x.lower()) + "+" + df_municipios['str_municipio'].apply(lambda x: unidecode.unidecode(str(re.sub(' +', ' ', x.strip())).lower()).replace(' ','-').replace('\'',''))
df_municipios['id_municipio'] = df_municipios['id_municipio'].astype(dtype = 'object')

# Salvando
UtilsPSQL().update_table('imoveis_municipios','replace', df_municipios)

5570 registros foram salvos na tabela imoveis_municipios do banco de dados scraping!


# Salvando como Parquet

In [34]:
# Diretório atual
current_dir = os.getcwd()

# Path do diretório para salvar dos dados
data_dir = os.path.join(current_dir, 'data', 'bronze')

# Criando o diretório caso nao exista
os.makedirs(data_dir, exist_ok  = True)

# Path do arquivo
parquet_file_path = os.path.join(data_dir, 'municipios_raw.parquet')

# Salvando DataFrame como parquet
df_municipios.to_parquet(parquet_file_path, engine = 'pyarrow')

# Print the file path (for verification)
print(f'DataFrame salvo como Parquet em: {parquet_file_path}')

DataFrame salvo como Parquet em: g:\Otros ordenadores\Meu modelo Laptop\Ciência de Dados\8. Projetos\Web_Scraping_Imoveis\app\data\bronze\municipios_raw.parquet


In [7]:
df_mun = pd.read_parquet(
    (os.path.join(os.getcwd(),'data','bronze','municipios_raw.parquet') if os.getcwd().__contains__('app') else os.path.join(os.getcwd(),'app','data','bronze','municipios_raw.parquet').replace('\\','/'))
)



df_mun.sort_values(by = ['str_uf', 'str_local'], ascending = [True, True], inplace = True)

locais = [x for x in list(map(lambda x: x ,df_mun['str_local'])) if x in list(df_imo['local'].unique())]

['ac+rio-branco',
 'al+maceio',
 'am+manaus',
 'ap+macapa',
 'ba+salvador',
 'ce+fortaleza',
 'df+brasilia',
 'es+vitoria',
 'go+goiania',
 'go+goias',
 'ma+sao-luis',
 'mg+belo-horizonte',
 'ms+campo-grande',
 'mt+cuiaba',
 'pa+belem',
 'pb+joao-pessoa',
 'pe+recife',
 'pi+teresina',
 'pr+curitiba',
 'pr+londrina',
 'rj+rio-de-janeiro',
 'rn+natal',
 'ro+porto-velho',
 'rr+boa-vista',
 'rs+porto-alegre',
 'sc+balneario-camboriu',
 'sc+florianopolis',
 'se+aracaju',
 'sp+campinas',
 'sp+guarulhos',
 'sp+osasco',
 'sp+sao-paulo',
 'to+palmas']